
**Package Installation**

In [ ]:
!git clone https://github.com/devkosal/fastai_roberta.git

Cloning into 'fastai_roberta'...
remote: Enumerating objects: 180, done.
remote: Counting objects: 100% (180/180), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 180 (delta 96), reused 110 (delta 44), pack-reused 0
Receiving objects: 100% (180/180), 25.46 MiB | 24.76 MiB/s, done.
Resolving deltas: 100% (96/96), done.


In [ ]:
!pip install fastai==1.0.60 transformers==2.3.0

     |████████████████████████████████| 245kB 18.7MB/s 
     |████████████████████████████████| 450kB 45.9MB/s 
     |████████████████████████████████| 133kB 46.8MB/s 
     |████████████████████████████████| 890kB 52.5MB/s 
     |████████████████████████████████| 1.2MB 47.8MB/s 
     |████████████████████████████████| 71kB 8.4MB/s 
     |████████████████████████████████| 7.3MB 47.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=05bc9c527bc67307d1195fa3962f4e494845b6c0cfe37d22abd3fbf00634ba34
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
ERROR: botocore 1.20.19 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.
  Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


**Load And Set Configuration**

In [ ]:
from fastai.text import *
from fastai.metrics import *


In [ ]:
from transformers import RobertaTokenizer

In [ ]:
# Creating a config object to store task specific information
class Config(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, k, v)
    
    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)
        
config = Config(
    testing=True,
    seed = 2019,
    roberta_model_name='roberta-base', # can also be exchnaged with roberta-large 
    max_lr=1e-5,
    epochs=1,
    use_fp16=False,
    bs=4, 
    max_seq_len=256, 
    num_labels = 2,
    hidden_dropout_prob=.05,
    hidden_size=768, # 1024 for roberta-large
    start_tok = "<s>",
    end_tok = "</s>",
)

In [ ]:
df = pd.read_csv("/content/dataset.csv")

In [ ]:
df=df.drop(df.columns[[0,3,4]], axis=1)
for i in range(len(df['Labels Set'])):
    df['Labels Set'][i]=list(df['Labels Set'][i].split(','))

In [ ]:
df

,Post,Labels Set
0,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्क...,"[hate, offensive]"
1,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,[non-hostile]
2,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील...",[non-hostile]
3,@prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं...,"[defamation, offensive]"
4,#unlock4guidelines - अनलॉक-4 के लिए गाइडलाइन्स...,[non-hostile]
...,...,...
5723,उदितराज जी हिम्मत जुटा कर उस नेता के लिए कुछ ...,[hate]
5724,उप्र: पीएम रिपोर्ट में खुलासा: हार्टअटैक से हु...,[non-hostile]
5725,नौकरी गंवा चुके दोस्त की मदद: नगद के बजाए गिफ्...,[non-hostile]
5726,बंगाल में हिन्दू मरे हैं इसलिए मुझे कोई फर्क न...,[fake]


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
multilabel = MultiLabelBinarizer()
y = multilabel.fit_transform(df['Labels Set'])
train_result=pd.DataFrame(y, columns=multilabel.classes_)
train_result = train_result.rename({'non-hostile': 'nh'}, axis=1)
train_result

,defamation,fake,hate,nh,offensive
0,0,0,1,0,1
1,0,0,0,1,0
2,0,0,0,1,0
3,1,0,0,0,1
4,0,0,0,1,0
...,...,...,...,...,...
5723,0,0,1,0,0
5724,0,0,0,1,0
5725,0,0,0,1,0
5726,0,1,0,0,0


In [ ]:
df = pd.concat([df, train_result], axis=1)

In [ ]:
df.head()

,Post,Labels Set,defamation,fake,hate,nh,offensive
0,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्क...,"[hate, offensive]",0,0,1,0,1
1,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,[non-hostile],0,0,0,1,0
2,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील...",[non-hostile],0,0,0,1,0
3,@prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं...,"[defamation, offensive]",1,0,0,0,1
4,#unlock4guidelines - अनलॉक-4 के लिए गाइडलाइन्स...,[non-hostile],0,0,0,1,0


In [ ]:
test_df = pd.read_csv("/content/validation.csv")
test_df=test_df.drop(test_df.columns[[0]], axis=1)
for i in range(len(test_df['Labels Set'])):
    test_df['Labels Set'][i]=list(test_df['Labels Set'][i].split(','))
from sklearn.preprocessing import MultiLabelBinarizer
multilabel = MultiLabelBinarizer()
y = multilabel.fit_transform(test_df['Labels Set'])
test_result=pd.DataFrame(y, columns=multilabel.classes_)
test_result = test_result.rename({'non-hostile': 'nh'}, axis=1)
test_result

,defamation,fake,hate,nh,offensive
0,0,0,0,1,0
1,1,0,0,0,0
2,0,0,0,1,0
3,0,0,0,1,0
4,0,0,0,1,0
...,...,...,...,...,...
806,0,0,1,0,0
807,1,0,0,0,0
808,0,0,0,1,0
809,0,0,0,1,0


In [ ]:
test_df = pd.concat([test_df, test_result], axis=1)
test_df.head()

,Post,Labels Set,defamation,fake,hate,nh,offensive
0,दृढ़ इच्छा शक्ति से परिपूर्ण प्रणबदा के लिए दे...,[non-hostile],0,0,0,1,0
1,भारतीय जनता पार्टी rss वाले इतने गिरे हुए हैं ...,[defamation],1,0,0,0,0
2,कोरोना से निपटने की तैयारी / दिल्ली में 10 हजा...,[non-hostile],0,0,0,1,0
3,गवर्नर कॉन्फ्रेंस में PM मोदी बोले- शिक्षा नीत...,[non-hostile],0,0,0,1,0
4,"यूपी: गाजीपुर में Toilet घोटाला, प्रधान व सचिव...",[non-hostile],0,0,0,1,0


In [ ]:
# frames = [df, test_df]
# df = pd.concat(frames)


In [ ]:
df.head()
df

,Post,Labels Set,defamation,fake,hate,nh,offensive
0,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्क...,"[hate, offensive]",0,0,1,0,1
1,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,[non-hostile],0,0,0,1,0
2,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील...",[non-hostile],0,0,0,1,0
3,@prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं...,"[defamation, offensive]",1,0,0,0,1
4,#unlock4guidelines - अनलॉक-4 के लिए गाइडलाइन्स...,[non-hostile],0,0,0,1,0
...,...,...,...,...,...,...,...
5723,उदितराज जी हिम्मत जुटा कर उस नेता के लिए कुछ ...,[hate],0,0,1,0,0
5724,उप्र: पीएम रिपोर्ट में खुलासा: हार्टअटैक से हु...,[non-hostile],0,0,0,1,0
5725,नौकरी गंवा चुके दोस्त की मदद: नगद के बजाए गिफ्...,[non-hostile],0,0,0,1,0
5726,बंगाल में हिन्दू मरे हैं इसलिए मुझे कोई फर्क न...,[fake],0,1,0,0,0


In [ ]:
feat_cols = "Post"
label_cols = "fake"


**Setting Up the Tokenizer**

In [ ]:

class FastAiRobertaTokenizer(BaseTokenizer):
    """Wrapper around RobertaTokenizer to be compatible with fastai"""
    def __init__(self, tokenizer: RobertaTokenizer, max_seq_len: int=128, **kwargs): 
        self._pretrained_tokenizer = tokenizer
        self.max_seq_len = max_seq_len 
    def __call__(self, *args, **kwargs): 
        return self 
    def tokenizer(self, t:str) -> List[str]: 
        """Adds Roberta bos and eos tokens and limits the maximum sequence length""" 
        return [config.start_tok] + self._pretrained_tokenizer.tokenize(t)[:self.max_seq_len - 2] + [config.end_tok]

In [ ]:
# create fastai tokenizer for roberta
roberta_tok = RobertaTokenizer.from_pretrained("roberta-base")

fastai_tokenizer = Tokenizer(tok_func=FastAiRobertaTokenizer(roberta_tok, max_seq_len=config.max_seq_len), 
                             pre_rules=[], post_rules=[])

In [ ]:
# create fastai vocabulary for roberta
path = Path()
roberta_tok.save_vocabulary(path)

with open('vocab.json', 'r') as f:
    roberta_vocab_dict = json.load(f)
    
fastai_roberta_vocab = Vocab(list(roberta_vocab_dict.keys()))

In [ ]:

# Setting up pre-processors
class RobertaTokenizeProcessor(TokenizeProcessor):
    def __init__(self, tokenizer):
         super().__init__(tokenizer=tokenizer, include_bos=False, include_eos=False)

class RobertaNumericalizeProcessor(NumericalizeProcessor):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)


def get_roberta_processor(tokenizer:Tokenizer=None, vocab:Vocab=None):
    """
    Constructing preprocessors for Roberta
    We remove sos and eos tokens since we add that ourselves in the tokenizer.
    We also use a custom vocabulary to match the numericalization with the original Roberta model.
    """
    return [RobertaTokenizeProcessor(tokenizer=tokenizer), RobertaNumericalizeProcessor(vocab=vocab)]

**Setting up the DataBunch**

In [ ]:
# Creating a Roberta specific DataBunch class
class RobertaDataBunch(TextDataBunch):
    "Create a `TextDataBunch` suitable for training Roberta"
    @classmethod
    def create(cls, train_ds, valid_ds, test_ds=None, path:PathOrStr='.', bs:int=64, val_bs:int=None, pad_idx=1,
               pad_first=True, device:torch.device=None, no_check:bool=False, backwards:bool=False, 
               dl_tfms:Optional[Collection[Callable]]=None, **dl_kwargs) -> DataBunch:
        "Function that transform the `datasets` in a `DataBunch` for classification. Passes `**dl_kwargs` on to `DataLoader()`"
        datasets = cls._init_ds(train_ds, valid_ds, test_ds)
        val_bs = ifnone(val_bs, bs)
        collate_fn = partial(pad_collate, pad_idx=pad_idx, pad_first=pad_first, backwards=backwards)
        train_sampler = SortishSampler(datasets[0].x, key=lambda t: len(datasets[0][t][0].data), bs=bs)
        train_dl = DataLoader(datasets[0], batch_size=bs, sampler=train_sampler, drop_last=True, **dl_kwargs)
        dataloaders = [train_dl]
        for ds in datasets[1:]:
            lengths = [len(t) for t in ds.x.items]
            sampler = SortSampler(ds.x, key=lengths.__getitem__)
            dataloaders.append(DataLoader(ds, batch_size=val_bs, sampler=sampler, **dl_kwargs))
        return cls(*dataloaders, path=path, device=device, dl_tfms=dl_tfms, collate_fn=collate_fn, no_check=no_check)

In [ ]:
class RobertaTextList(TextList):
    _bunch = RobertaDataBunch
    _label_cls = TextList

In [ ]:
# loading the tokenizer and vocab processors
processor = get_roberta_processor(tokenizer=fastai_tokenizer, vocab=fastai_roberta_vocab)

In [ ]:
# creating our databunch 
data = RobertaTextList.from_df(df, ".", cols=feat_cols, processor=processor) \
    .split_by_rand_pct(seed=config.seed) \
    .label_from_df(cols=label_cols,label_cls=CategoryList) \
    .databunch(bs=config.bs, pad_first=False, pad_idx=0)

/usr/local/lib/python3.7/dist-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(a, dtype=dtype, **kwargs)


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:

data

RobertaDataBunch;

Train: LabelList (4583 items)
x: RobertaTextList
<s> à¤ ® à¥ ĩ à¤ ° à¥ ĩ Ġà¤ ¦ à¥ ĩ à¤ ¶ Ġà¤ ķ à¥ ĩ Ġà¤ ¹ à¤ ¿ à¤ ¨ à¥ į à¤ ¦ à¥ ģ Ġà¤ ¬ à¤ ¹ à¥ ģ à¤ ¤ Ġà¤ ¨ à¤ ¿ à¤ ° à¤¾ à¤ ² à¥ ĩ Ġà¤ ¹ à¥ Ī à¥ ¤ Ġà¤ ķ à¥ ģ à¤ Ľ Ġà¤ ¤ à¥ ĭ Ġà¤ ª à¤ ķ à¥ į à¤ ķ à¥ ĩ Ġà¤ ° à¤¾ à¤ ® Ġà¤ Ń à¤ ķ à¥ į à¤ ¤ Ġà¤ ¹ à¥ Ī Ġà¤ Ķ à¤ ° Ġà¤ ķ à¥ ģ à¤ Ľ Ġà¤ ¬ à¤¾ à¤ ¬ à¤ ° Ġà¤ ķ à¥ ĩ Ġà¤ ¸ à¤¾ à¤ ² à¥ ĩ Ġà¤ ¹ à¥ Ī Ġ Ċ Ċ ðŁ Ļ ı Ġà¤ ľ à¤ ¯ Ġà¤ ¶ à¥ į à¤ ° à¥ Ģ Ġà¤ ° à¤¾ à¤ ® ĠðŁ Ļ ı </s>,<s> à¤ ¸ à¤ ° à¤ ķ à¤¾ à¤ ° Ġà¤ ¹ à¤ ® à¥ ĩ à¤ ¶ à¤¾ Ġà¤ ¸ à¥ ĩ Ġà¤ ķ à¤ ¿ à¤ ¸ à¤¾ à¤ ¨ à¥ ĭ à¤ Ĥ Ġà¤ ķ à¥ Ģ Ġà¤ ķ à¤ ® à¤¾ à¤ Ī Ġà¤ ķ à¥ ĭ Ġà¤ ¬ à¤ ¢ à¤ ¼ à¤¾ à¤ ¨ à¥ ĩ Ġà¤ ķ à¥ ĩ Ġà¤ ² à¤ ¿ à¤ ı Ġà¤ ¨ à¤ Ī - à¤ ¨ à¤ Ī Ġà¤ ¸ à¥ į à¤ ķ à¥ Ģ à¤ ® à¥ ĩ à¤ Ĥ Ġà¤ ² à¤¾ à¤ ¤ à¥ Ģ Ġà¤ ° à¤ ¹ à¤ ¤ à¥ Ģ Ġà¤ ¹ à¥ Ī , Ġà¤ ¤ à¤¾ à¤ ķ à¤ ¿ Ġà¤ ī à¤ ¨ Ġà¤ ª à¤ ° Ġà¤ ľ à¥ į à¤ ¯ à¤¾ à¤ ¦ à¤¾ Ġà¤ Ĩ à¤ ° à¥ į à¤ ¥ à¤ ¿ à¤ ķ Ġà¤ ¬ à¥ ĭ à¤ Ŀ Ġà¤ ¨ Ġà¤ ª à¤ ¡ à¤ ¼ à¥ ĩ . Ċ Ċ https :// t . co / 8 iy 2 MJ SB As </s>,<s> @ pr ab h av 21

**Building the Model** 

In [ ]:
import torch
import torch.nn as nn
from transformers import RobertaModel

# defining our model architecture 
class CustomRobertaModel(nn.Module):
    def __init__(self,num_labels=2):
        super(CustomRobertaModel,self).__init__()
        self.num_labels = num_labels
        self.roberta = RobertaModel.from_pretrained(config.roberta_model_name)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, num_labels) # defining final output layer
        
    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        _ , pooled_output = self.roberta(input_ids, token_type_ids, attention_mask) # 
        logits = self.classifier(pooled_output)        
        return logits

In [ ]:

roberta_model = CustomRobertaModel(num_labels=config.num_labels)

learn = Learner(data, roberta_model, metrics=[accuracy])

In [ ]:
learn.model.roberta.train() # setting roberta to train as it is in eval mode by default
learn.fit_one_cycle(config.epochs, max_lr=config.max_lr)

epoch,train_loss,valid_loss,accuracy,time
0,0.367236,0.337991,0.806114,04:09


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
def get_preds_as_nparray(ds_type) -> np.ndarray:
    learn.model.roberta.eval()
    preds = learn.get_preds(ds_type)[0].detach().cpu().numpy()
    sampler = [i for i in data.dl(ds_type).sampler]
    reverse_sampler = np.argsort(sampler)
    ordered_preds = preds[reverse_sampler, :]
    pred_values = np.argmax(ordered_preds, axis=1)
    return ordered_preds, pred_values

In [ ]:
preds, pred_values = get_preds_as_nparray(DatasetType.Valid)


In [ ]:
# accuracy on valid
(pred_values == data.valid_ds.y.items).mean()

0.8061135371179039

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(data.valid_ds.y.items,pred_values))

              precision    recall  f1-score   support

           0       0.81      1.00      0.89       923
           1       0.00      0.00      0.00       222

    accuracy                           0.81      1145
   macro avg       0.40      0.50      0.45      1145
weighted avg       0.65      0.81      0.72      1145



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
